In [1]:
#include "Math/GenVector/PxPyPzE4D.h"
#include "Math/GenVector/DisplacementVector3D.h"
#include "Math/GenVector/GenVectorIO.h"
#include "Math/GenVector/LorentzVector.h"
#include "TFile.h"
#include "TTree.h"

In [2]:
typedef ROOT::Math::LorentzVector<ROOT::Math::PxPyPzE4D<double> > XYZTLorentzVectorD;
typedef XYZTLorentzVectorD XYZTLorentzVector;

In [3]:
Double_t DeltaPhi(XYZTLorentzVector a,XYZTLorentzVector b){
    Double_t deltaphi = (a.Phi()-b.Phi());
    if (deltaphi >= 3.14159265) deltaphi -= 2*3.14159265;
    if (deltaphi < -3.14159265) deltaphi += 2*3.14159265;
    return deltaphi;
}

In [4]:
std::string FilePath = "/home/camilo/cms/VLF_Montecarlo_Ana/Data/";

//std::string FileName = "Signal.root";

//std::string FileName = "DYJetsToLL_HT/DYJetsToLL_HT.root";

//std::string FileName = "ST/ST_t-channel_top_4f_inclusiveDecays.root";
//std::string FileName = "ST/ST_s-channel_4f_leptonDecays.root";
//std::string FileName = "ST/ST_t-channel_antitop_4f_inclusiveDecays.root";
//std::string FileName = "ST/ST_tW_antitop_5f_inclusiveDecays.root";
//std::string FileName = "ST/ST_tW_top_5f_inclusiveDecays.root";

//std::string FileName = "TTToLL_MLL/TT_TuneCUETP8M2T4.root";

//std::string FileName = "WJets_HT/WJetsToLNu_HT-70To100.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-100To200.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-200To400.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-400To600.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-600To800.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-800To1200.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-1200To2500.root";
std::string FileName = "WJets_HT/WJetsToLNu_HT-2500ToInf.root";

std::string File = FilePath+FileName;
std::string PathTree = "demo/VLFTree";

In [5]:
TFile *f = new TFile(File.c_str(),"update"); //Open File
TTree *Tree = (TTree*)f->Get(PathTree.c_str()); //Get the Tree
int nevent = Tree->GetEntries();
//Int_t count = 0;

In [6]:
std::vector<XYZTLorentzVector> *AnaMuons = new std::vector<XYZTLorentzVector>();
Tree->SetBranchAddress("AnaMuons", &AnaMuons);
XYZTLorentzVector * MET = new XYZTLorentzVector();
Tree->SetBranchAddress("AnaMET", &MET);
vector<double> *Charge;
Tree->SetBranchAddress("AnaMuon_charge", &Charge);
//std::vector <bool> *Muon_Medium;
//Tree->SetBranchAddress("AllMuonMediumID", &Muon_Medium);
//std::vector <double> *Combined_Iso;
//Tree->SetBranchAddress("AllCombined_iso_DeltaBetaPU", &Combined_Iso);

In [7]:
Bool_t debug = kFALSE;
//debug = kTRUE;

In [8]:
//NEW Branches
std::vector<XYZTLorentzVector> *TheMuon = new std::vector<XYZTLorentzVector>();
TBranch *TheMuonB = Tree->Branch("TheMuon",&TheMuon); 
std::vector<Double_t> MT_TheMuon_MET;
TBranch *MT_TheMuon_METB = Tree->Branch("MT_TheMuon_MET",&MT_TheMuon_MET); 
Double_t IMass;
TBranch *IMassB = Tree->Branch("IMass",&IMass); 

In [9]:
//TBranch *TheMuonB = Tree->Branch("TheMuon",&TheMuon); 

In [10]:
if(debug) nevent = 1000;
for (int i = 0 ; i < nevent ; i++ ) {
    if(debug)std::cout<<"Event: "<< i <<std::endl;
    XYZTLorentzVector  TheMuon1;
    Tree->GetEvent(i); 
    if(AnaMuons->size()==2){
        Double_t MT_LeadingMuon_MET1 = sqrt(2*AnaMuons->at(0).Pt()*MET->Pt()*(1-cos(DeltaPhi(AnaMuons->at(0),*MET))));
        Double_t MT_LeadingMuon_MET2 = sqrt(2*AnaMuons->at(1).Pt()*MET->Pt()*(1-cos(DeltaPhi(AnaMuons->at(1),*MET))));
        
        if ( MT_LeadingMuon_MET1 >= MT_LeadingMuon_MET2){
            
            TheMuon1.SetPxPyPzE(AnaMuons->at(0).px(), AnaMuons->at(0).py(), AnaMuons->at(0).pz(), AnaMuons->at(0).energy());
            MT_TheMuon_MET.push_back(MT_LeadingMuon_MET1);
            TheMuon->push_back(TheMuon1);
            TheMuon1.SetPxPyPzE(AnaMuons->at(1).px(), AnaMuons->at(1).py(), AnaMuons->at(1).pz(), AnaMuons->at(1).energy());
            MT_TheMuon_MET.push_back(MT_LeadingMuon_MET2);
            TheMuon->push_back(TheMuon1);
            
            if(debug){
                std::cout<< "Mayor1 p_T: " <<TheMuon->at(0).pt() <<std::endl;
                std::cout<< "Menor2 p_T: " <<TheMuon->at(1).pt() <<std::endl;
                std::cout<< "Mayor1 m_T: " <<MT_TheMuon_MET.at(0) <<std::endl;
                std::cout<< "Menor2 m_T: " <<MT_TheMuon_MET.at(1) <<std::endl;
            }
        }
        
      
        
        if ( MT_LeadingMuon_MET1 < MT_LeadingMuon_MET2){
            //XYZTLorentzVector  TheMuon1;
            TheMuon1.SetPxPyPzE(AnaMuons->at(1).px(), AnaMuons->at(1).py(), AnaMuons->at(1).pz(), AnaMuons->at(1).energy());
            MT_TheMuon_MET.push_back(MT_LeadingMuon_MET2);
            TheMuon->push_back(TheMuon1);
            TheMuon1.SetPxPyPzE(AnaMuons->at(0).px(), AnaMuons->at(0).py(), AnaMuons->at(0).pz(), AnaMuons->at(0).energy());
            MT_TheMuon_MET.push_back(MT_LeadingMuon_MET1);
            TheMuon->push_back(TheMuon1);
            if(debug){
                std::cout<< "Mayor2 p_T: " <<TheMuon->at(0).pt() <<std::endl;
                std::cout<< "Menor1 p_T: " <<TheMuon->at(1).pt() <<std::endl;
                std::cout<< "Mayor2 m_T: " <<MT_TheMuon_MET.at(0) <<std::endl;
                std::cout<< "Menor1 m_T: " <<MT_TheMuon_MET.at(1) <<std::endl;
        
            }
        }
        
        //Invariant Mass
        if(debug) std::cout<< "Charges: "<< Charge->at(0)<< ", "<< Charge->at(1) <<std::endl;
        if((Int_t)Charge->at(0)*(Int_t)Charge->at(1) < 0 ){
            IMass = (AnaMuons->at(0)+AnaMuons->at(1)).M();
            if(debug) std::cout<< "Invariant Mass Two Muons: "<< IMass <<std::endl;
        }else if(debug) std::cout<< "Same charge-IMass: "<< IMass <<std::endl;
        
        
    }else if(AnaMuons->size()==1){
        Double_t MT_LeadingMuon_MET1 = sqrt(2*AnaMuons->at(0).Pt()*MET->Pt()*(1-cos(DeltaPhi(AnaMuons->at(0),*MET))));

        //XYZTLorentzVector  TheMuon1;
        TheMuon1.SetPxPyPzE(AnaMuons->at(0).px(), AnaMuons->at(0).py(), AnaMuons->at(0).pz(), AnaMuons->at(0).energy());
            MT_TheMuon_MET.push_back(MT_LeadingMuon_MET1);
            TheMuon->push_back(TheMuon1);
            if(debug){
                std::cout<< "Only Muon p_T: " <<TheMuon->at(0).pt() <<std::endl;
                std::cout<< "Only Muon m_T: " <<MT_TheMuon_MET.at(0) <<std::endl;
            }
        //Invariant Mass
        IMass = -1;
        if(debug) std::cout<< "Invariant Mass One Muon: "<< IMass <<std::endl;
        
        
    }else{
        std::cout<< "ERROR: Not allowed Muon number" <<std::endl;
        std::cout<< "ERROR: Not allowed Muon number" <<std::endl;
        IMass=-1;
    }
        if(!debug)TheMuonB->Fill();
        if(!debug)MT_TheMuon_METB->Fill();
        if(!debug)IMassB->Fill();
        IMass=0;
        MT_TheMuon_MET.clear();
        TheMuon->clear();
}
if(!debug)Tree->Write(); 

In [11]:
f->Close();